In [5]:
import pickle
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import log_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from scipy.stats import randint as sp_randint

%matplotlib inline
sns.set(style="whitegrid")

import os; os.chdir('..')
import rental_utils; reload(rental_utils)

<module 'rental_utils' from 'rental_utils.pyc'>

In [6]:
data, independent, test = rental_utils.load_data()
[X_train, X_val, y_train, y_val] = data

Loading raw data
Adding features
Adding regions
Adding variables
Dummyfying
Splitting data


In [7]:
clf = RandomForestClassifier()

param_dist = {"n_estimators": sp_randint(50,100),
              "max_depth": sp_randint(10,100),
              "max_features": [0.1, 0.2, 0.33, 0.5],
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

n_iter_search = 10
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, verbose=1)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  7.5min finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000003F5C67F0>, 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000003FB70470>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000003F5C6898>, 'criterion': ['gini', 'entropy'], 'max_features': [0.1, 0.2, 0.33, 0.5], 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000

In [11]:
pd.DataFrame(random_search.best_params_.items(), columns=['parameter', 'value'])

,parameter,value
0,bootstrap,True
1,min_samples_leaf,2
2,n_estimators,51
3,min_samples_split,9
4,criterion,entropy
5,max_features,0.33
6,max_depth,87


In [12]:
model = random_search.best_estimator_
preds, probs = rental_utils.predict(model, X_val)
print('Log loss:            ' + str(round(log_loss(y_val, probs), 3)))

Log loss:            0.622


In [15]:
pickle.dump(model, open("Models/best_rf.pkl", "wb"))
model2 = pickle.load(open("Models/best_rf.pkl", "rb"))